<a href="https://colab.research.google.com/github/Bart-del/studies-machine-learning/blob/main/Zadanie_5_B_wysocki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Zadanie 5

Plik gratkapl-centrenrm.csv zawiera dane wzbogacone o nową cechę Centre, przyjmującą wartość 1, jeżeli lokalizacja mieszkania to centrum, i 0 w przeciwnym przypadku.

Zamiast przywidywania ceny mieszkania, spróbuj przewidzieć lokalizację mieszkania: centrum vs. nie-centrum za pomocą regresji logistycznej.

    Podziel zbiór danych na dane treningowe (pierwsze 4/5 danych) i testowe (ostatnie 1/5 danych)
    Stwórz dwuklasowy model regresji logistycznej, korzystając z cech liczbowych Price,Rooms,SqrMeters,Floor. Warto usunąć obserwacje odstające i dokonać normalizacji danych.
    Podaj procent poprawnych wyników (tzw. skuteczność, accuracy) dla zestawu testowego.

Uwaga: Cecha Centre została wygenerowana automatycznie na podstawie opisów lokalizacji mieszkań i okazuje się, że jest słabo skorelowana z cechami liczbowymi, dlatego proszę się nie przejmować, jeżeli wartości funkcji regresji logistycznej i przewidywania modelu będą nieco dziwne.

Wskazówka: nie trzeba ręcznie implementować metody gradientu prostego - można skorzystać z gotowych modeli, np. z pakietu scikit-learn.

<h2>Wczytanie pliku z danymi

In [1]:
from google.colab import files
uploaded = files.upload()

Saving gratkapl-centrenrm.csv to gratkapl-centrenrm.csv


<h2>Zaimportowanie bibliotek i stworzenie DataFrame

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import csv

from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler


reader = csv.reader(open('gratkapl-centrenrm.csv', encoding='utf-8'), delimiter=',')
data = pd.DataFrame(reader)
new_header = data.iloc[0]
data = data[1:]
data.columns = new_header
data

,Id,Price,Rooms,SqrMeters,Floor,Location,Centre,Description
1,3914,386064,2,46,3,Poznań Stare Miasto Wilda ul. Maratońska Łęgi ...,1,OFERTA ZWOLNIONA Z PROWIZJI OD KUPUJĄCEGO! OFE...
2,437,280000,2,41,5,Poznań Poznań-Wilda,0,Polecam do sprzedaży mieszkanie dwupokojowe pr...
3,1139,329000,2,50.5,1,Poznań Ogrody,0,Polecam na sprzedaż mieszkanie 2-pokojowe na p...
4,4377,225000,2,43,10,Poznań Grunwald Łazarz Górczyn Ogrody Je ul. N...,1,BEZPIECZNA TRANSAKCJA JEDYNIE Z DOŚWIADCZONYM ...
5,192,280000,2,69,3,Poznań Wilda ul. Traugutta,0,Salon nieruchomości Iron House poleca na sprze...
...,...,...,...,...,...,...,...,...
2014,128,277000,2,46.1,3,Poznań Naramowice Piątkowo Winogrady ul. Sielawy,0,Kupując z pośrednikiem - kupujesz bezpiecznie ...
2015,484,371000,3,47,2,Poznań Łazarz ul. Grottgera 15/19,0,Najbardziej Prestiżowy Adres w Poznaniu!! Prze...
2016,795,341600,2,61,1,Poznań Poznań-Grunwald ul. Jana Matejki,0,Oferta sprzedaży mieszkania o łącznej powierzc...
2017,1562,149000,1,28,3,Poznań Dębiec,0,Kawalerka 28m2 na Dębcu SUPER CENA !!!! Poleca...


<h2> Usunięcie niepotrzebnych danych, usunięcie wartości odstających

In [18]:
del data["Location"]
del data["Description"]
del data["Id"]

data['Price'] = pd.to_numeric(data['Price'])
data['Centre'] = pd.to_numeric(data['Centre'])
data['Rooms'] = pd.to_numeric(data['Rooms'])
data['Floor'] = pd.to_numeric(data['Floor'])
data['SqrMeters'] = pd.to_numeric(data['SqrMeters'])

data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]

<h2>Normalizacja danych

In [24]:
data

,Price,Rooms,SqrMeters,Floor,Centre
1,0.250853,0.25,0.273309,0.222222,1.0
2,0.181936,0.25,0.230105,0.444444,0.0
3,0.213775,0.25,0.312192,0.000000,0.0
4,0.146198,0.25,0.247386,1.000000,1.0
5,0.181936,0.25,0.472047,0.222222,0.0
...,...,...,...,...,...
2014,0.179986,0.25,0.274173,0.222222,0.0
2015,0.241065,0.50,0.281949,0.111111,0.0
2016,0.221962,0.25,0.402921,0.000000,0.0
2017,0.096816,0.00,0.117774,0.222222,0.0


In [27]:
scaler = MinMaxScaler()
data[['Price', 'Rooms', 'SqrMeters', 'Floor', 'Centre']] = scaler.fit_transform(data[['Price', 'Rooms', 'SqrMeters', 'Floor', 'Centre']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3076: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc._setitem_with_indexer((slice(None), indexer), value)
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3041: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

<h2>Podzielenie danych na zbiór uczący i testowy

In [29]:
trening_set = data[:1550]
test_set = data[1550:]

X_trening = trening_set[['Rooms', 'SqrMeters', 'Floor', 'Centre']]
y_trening = trening_set[['Price']]

X_test = test_set[['Rooms', 'SqrMeters', 'Floor', 'Centre']]
y_test = test_set[['Price']]

<h2>Tworzenie modelu

In [ ]:
classifer = LogisticRegression()